In [5]:
%load_ext autoreload
from ddls.distributions.custom_skew_norm import CustomSkewNorm
from ddls.distributions.uniform import Uniform
from ddls.utils import seed_stochastic_modules_globally
from ddls.environments.ramp_cluster.utils import load_ramp_cluster_environment_metrics_from_wandb_run
from ddls.environments.ramp_cluster.utils import load_ramp_cluster_environment_metrics_from_wandb_sweep

from ddls.plotting.plotting import PlotAesthetics

from collections import defaultdict
from sqlitedict import SqliteDict
import pprint
import glob
import os
import time
import pandas as pd
import numpy as np
import scipy.stats as st
import sigfig

import gzip
import pickle

import copy

from omegaconf import OmegaConf
from functools import reduce

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
%autoreload

learner_agent_to_beta_dist_to_run_path = {
    
    'PAC-ML (Ours)': {
        
            '$\\beta_{A}$': 'ong/ddls/x08ia9ap',
            '$\\beta_{B}$': 'ong/ddls-scripts/nhts5bxs',
            '$\\beta_{C}$': 'ong/ddls/1r207clu',
            '$\\beta_{D}$': 'ong/ddls/2np29sr3',
        
    },
    
}



baseline_agent_to_beta_dist_to_sweep_path = {
    
    'Para$_{max}$': {
        
            '$\\beta_{A}$': 'ong/ddls-scripts/xprqeqoh',
            '$\\beta_{B}$': 'ong/ddls-scripts/39cz3lq6',
            '$\\beta_{C}$': 'ong/ddls-scripts/j627d02o',
            '$\\beta_{D}$': 'ong/ddls-scripts/90nahn4v',
        
    },
    
    'Para$_{min}$': {
        
            '$\\beta_{A}$': 'ong/ddls-scripts/lv27j5ia',
            '$\\beta_{B}$': 'ong/ddls-scripts/abxkd48j',
            '$\\beta_{C}$': 'ong/ddls-scripts/olttukv3',
            '$\\beta_{D}$': 'ong/ddls-scripts/4e5i2xy1',
        
    },
    
    'Random': {
        
            '$\\beta_{A}$': 'ong/ddls-scripts/4f1n6nxa',
            '$\\beta_{B}$': 'ong/ddls-scripts/9xz174fq',
            '$\\beta_{C}$': 'ong/ddls-scripts/x6ber86a',
            '$\\beta_{D}$': 'ong/ddls-scripts/3h0qd3ub',
        
    },
    
}

In [3]:
%autoreload

# load learner agent run
keys_to_ignore = ['training/'] # for ignoring data not needed for plotting
key_substrings_to_remove = ['evaluation/'] # for tidying up keys
verbose = True

_learner_agent_to_episode_stats_dict, _learner_agent_to_episode_completion_stats_dict, _learner_agent_to_episode_blocked_stats_dict = defaultdict(list), defaultdict(list), defaultdict(list)
for learner_agent in learner_agent_to_beta_dist_to_run_path.keys():
    # load agent data for each dist
    hue = '$\\beta_{X}$' # grouping by beta dist
    dist_to_run = learner_agent_to_beta_dist_to_run_path[learner_agent]
    learner_agent_to_episode_stats_dict, learner_agent_to_episode_completion_stats_dict, learner_agent_to_episode_blocked_stats_dict = load_ramp_cluster_environment_metrics_from_wandb_run(dist_to_run, keys_to_ignore=keys_to_ignore, key_substrings_to_remove=key_substrings_to_remove, verbose=verbose, hue=hue)
    
    # add agent column
    learner_agent_to_episode_stats_dict['Agent'].extend([learner_agent for _ in range(len(learner_agent_to_episode_stats_dict[hue]))])
    learner_agent_to_episode_completion_stats_dict['Agent'].extend([learner_agent for _ in range(len(learner_agent_to_episode_completion_stats_dict[hue]))])
    learner_agent_to_episode_blocked_stats_dict['Agent'].extend([learner_agent for _ in range(len(learner_agent_to_episode_blocked_stats_dict[hue]))])
    
    # HACK: Forgot to record epoch_counter in completion stats dict and blocked stats dict, need to add manually
    # NOTE: This is only rough approximation since in fact on e.g. first epoch, usually have no completed jobs so no rows added to completion stats, so epoch counters will not be
    # consistent across dataframes. Should only really use this hack if you are just wanting to index e.g. the last epoch rather than indexing specific epochs
    # across different dataframes (e.g. do not index the best epoch since will not be consistent across data frames)
    num_seeds = 3
    learner_agent_to_episode_completion_stats_dict['epoch_counter'].extend([epoch_counter for _ in range(num_seeds) for epoch_counter in range(int(len(learner_agent_to_episode_completion_stats_dict[hue]) / num_seeds))])
    if len(learner_agent_to_episode_completion_stats_dict['epoch_counter']) < len(learner_agent_to_episode_completion_stats_dict[hue]):
        # quit run before other seeds could finish, need to add final epoch seed(s) manually to make all columns in df the same length
        final_epoch = learner_agent_to_episode_completion_stats_dict['epoch_counter'][-1] + 1
        for _ in range(int(len(learner_agent_to_episode_completion_stats_dict[hue]) - len(learner_agent_to_episode_completion_stats_dict['epoch_counter']))):
            learner_agent_to_episode_completion_stats_dict['epoch_counter'].append(final_epoch)
    learner_agent_to_episode_blocked_stats_dict['epoch_counter'].extend([epoch_counter for _ in range(num_seeds) for epoch_counter in range(int(len(learner_agent_to_episode_blocked_stats_dict[hue]) / num_seeds))])
    if len(learner_agent_to_episode_blocked_stats_dict['epoch_counter']) < len(learner_agent_to_episode_blocked_stats_dict[hue]):
        # quit run before other seeds could finish, need to add final epoch seed(s) manually to make all columns in df the same length
        final_epoch = learner_agent_to_episode_blocked_stats_dict['epoch_counter'][-1] + 1
        for _ in range(int(len(learner_agent_to_episode_blocked_stats_dict[hue]) - len(learner_agent_to_episode_blocked_stats_dict['epoch_counter']))):
            learner_agent_to_episode_blocked_stats_dict['epoch_counter'].append(final_epoch)
            
    # merge learner agent df into all learner agents df
    for key, val in learner_agent_to_episode_stats_dict.items():
        _learner_agent_to_episode_stats_dict[key].extend(val)
    for key, val in learner_agent_to_episode_completion_stats_dict.items():
        _learner_agent_to_episode_completion_stats_dict[key].extend(val)
    for key, val in learner_agent_to_episode_blocked_stats_dict.items():
        _learner_agent_to_episode_blocked_stats_dict[key].extend(val)
            
# remove leading underscore for readability
learner_agent_to_episode_stats_dict, learner_agent_to_episode_completion_stats_dict, learner_agent_to_episode_blocked_stats_dict = _learner_agent_to_episode_stats_dict, _learner_agent_to_episode_completion_stats_dict, _learner_agent_to_episode_blocked_stats_dict

    
# convert dicts to dataframes
learner_agent_episode_stats_df = pd.DataFrame(learner_agent_to_episode_stats_dict)
display(learner_agent_episode_stats_df)

for key, val in learner_agent_to_episode_completion_stats_dict.items():
    print(f'key {key} -> {len(val)} vals')
learner_agent_episode_completion_stats_df = pd.DataFrame(learner_agent_to_episode_completion_stats_dict)
display(learner_agent_episode_completion_stats_df)

for key, val in learner_agent_to_episode_blocked_stats_dict.items():
    print(f'key {key} -> {len(val)} vals')
learner_agent_episode_blocked_stats_df = pd.DataFrame(learner_agent_to_episode_blocked_stats_dict)
display(learner_agent_episode_blocked_stats_df)


Recorded keys: {'evaluation/episode_time_min', 'evaluation/jobs_blocked_total_operation_memory_cost_max', 'evaluation/mean_compute_throughput_mean', 'evaluation/jobs_completed_max_partitions_per_op_min', 'evaluation/episode_counter', 'evaluation/mean_compute_throughput_max', 'evaluation/job_completion_time_min', 'evaluation/jobs_completed_num_mounted_channels_min', 'evaluation/mean_num_jobs_running_mean', 'evaluation/mean_inference_ms', 'evaluation/mean_cluster_worker_utilisation_frac_min', 'trackers/epoch_counter', 'evaluation/jobs_completed_original_demand_total_dependency_size_mean', 'evaluation/jobs_completed_max_acceptable_job_completion_time_min', 'evaluation/jobs_blocked_max_acceptable_job_completion_time_frac_max', 'evaluation/jobs_blocked_max_acceptable_job_completion_time_mean', 'evaluation/jobs_completed_max_acceptable_job_completion_time_max', 'evaluation/episode_end_time_mean', 'evaluation/mean_num_jobs_running_min', 'evaluation/job_completion_time_speedup_mean', 'evaluat

,mean_mounted_worker_utilisation_frac_mean,mean_mounted_worker_utilisation_frac_max,episode_reward_max,flow_info_processed_mean,mean_num_jobs_running_mean,mean_demand_compute_throughput_mean,dep_info_processed_mean,dep_info_processed_min,mean_cluster_worker_utilisation_frac_mean,num_jobs_blocked_max,...,num_jobs_arrived_max,compute_info_processed_mean,mean_demand_compute_throughput_min,demand_dep_info_processed_max,mean_communication_overhead_frac_mean,mean_demand_total_throughput_min,mean_load_rate_max,episode_start_time_min,$\beta_{X}$,Agent
0,0.000000e+00,0.000000e+00,-1000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1000,...,1001,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,3.162332e+07,0,$\beta_{A}$,PAC-ML (Ours)
1,0.000000e+00,0.000000e+00,-1000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1000,...,1001,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,3.156234e+07,0,$\beta_{A}$,PAC-ML (Ours)
2,0.000000e+00,0.000000e+00,-1000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1000,...,1001,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,3.159283e+07,0,$\beta_{A}$,PAC-ML (Ours)
3,0.000000e+00,0.000000e+00,-1000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1000,...,1001,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,3.161611e+07,0,$\beta_{A}$,PAC-ML (Ours)
4,0.000000e+00,0.000000e+00,-1000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1000,...,1001,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,3.159283e+07,0,$\beta_{A}$,PAC-ML (Ours)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125,4.922830e+06,7.328337e+06,846,1.392378e+13,13.367000,1.410547e+07,2.153358e+13,2.139938e+13,4.128312e+06,106,...,1001,1.410547e+13,1.401358e+07,1.446661e+13,0.000188,2.826119e+07,3.167275e+07,0,$\beta_{D}$,PAC-ML (Ours)
1126,4.940396e+06,8.418230e+06,828,1.382344e+13,13.317000,1.400386e+07,2.138129e+13,2.118714e+13,4.118171e+06,110,...,1001,1.400386e+13,1.389112e+07,1.431124e+13,0.000188,2.798920e+07,3.156234e+07,0,$\beta_{D}$,PAC-ML (Ours)
1127,4.946252e+06,7.899302e+06,812,1.383654e+13,13.258333,1.402108e+07,2.139056e+13,2.128791e+13,4.096123e+06,111,...,1001,1.402108e+13,1.396126e+07,1.429002e+13,0.000189,2.812903e+07,3.161404e+07,0,$\beta_{D}$,PAC-ML (Ours)
1128,4.926589e+06,7.380375e+06,826,1.392033e+13,13.350667,1.410006e+07,2.153123e+13,2.118138e+13,4.121842e+06,108,...,1001,1.410006e+13,1.388438e+07,1.445178e+13,0.000188,2.798197e+07,3.156876e+07,0,$\beta_{D}$,PAC-ML (Ours)


key job_completion_time_speedup_mean -> 1105 vals
key jobs_completed_total_dependency_size_max -> 1105 vals
key jobs_completed_max_acceptable_job_completion_time_mean -> 1105 vals
key job_communication_overhead_time_max -> 1105 vals
key jobs_completed_num_mounted_workers_min -> 1105 vals
key job_completion_time_speedup_min -> 1105 vals
key jobs_completed_max_partitions_per_op_mean -> 1105 vals
key jobs_completed_max_acceptable_job_completion_time_frac_mean -> 1105 vals
key jobs_completed_num_edges_max -> 1105 vals
key jobs_completed_max_acceptable_job_completion_time_frac_min -> 1105 vals
key jobs_completed_original_demand_total_operation_memory_cost_min -> 1105 vals
key jobs_completed_job_sequential_completion_time_mean -> 1105 vals
key jobs_completed_mean_mounted_worker_utilisation_frac_mean -> 1105 vals
key jobs_completed_total_operation_memory_cost_min -> 1105 vals
key jobs_completed_mean_mounted_worker_utilisation_frac_max -> 1105 vals
key job_completion_time_speedup_max -> 1105 v

,job_completion_time_speedup_mean,jobs_completed_total_dependency_size_max,jobs_completed_max_acceptable_job_completion_time_mean,job_communication_overhead_time_max,jobs_completed_num_mounted_workers_min,job_completion_time_speedup_min,jobs_completed_max_partitions_per_op_mean,jobs_completed_max_acceptable_job_completion_time_frac_mean,jobs_completed_num_edges_max,jobs_completed_max_acceptable_job_completion_time_frac_min,...,job_completion_time_min,job_completion_time_max,job_communication_overhead_time_mean,jobs_completed_total_dependency_size_mean,job_communication_overhead_time_min,jobs_completed_max_acceptable_job_completion_time_frac_max,jobs_completed_num_mounted_channels_max,$\beta_{X}$,Agent,epoch_counter
0,7.984287,1.366602e+11,33256.979314,8.528821,8.000000,7.984287,8.000000,0.963263,6474.000000,0.945455,...,4324.161851,4324.161851,8.528821,1.366602e+11,8.528821,0.983333,56.0,$\beta_{A}$,PAC-ML (Ours),0
1,7.984287,1.366602e+11,31992.614166,8.528821,8.000000,7.984287,8.000000,0.926641,6474.000000,0.909524,...,4324.161851,4324.161851,8.528821,1.366602e+11,8.528821,0.951600,56.0,$\beta_{A}$,PAC-ML (Ours),1
2,7.984287,1.366602e+11,31382.994961,8.528821,8.000000,7.984287,8.000000,0.908984,6474.000000,0.890244,...,4324.161851,4324.161851,8.528821,1.366602e+11,8.528821,0.926364,56.0,$\beta_{A}$,PAC-ML (Ours),2
3,7.874956,1.366602e+11,29513.856128,8.528821,7.764706,7.749798,7.890319,0.853856,6474.000000,0.832000,...,4324.161851,4872.823631,8.367708,1.339617e+11,8.170715,0.870156,56.0,$\beta_{A}$,PAC-ML (Ours),3
4,7.844884,1.366602e+11,28638.855700,8.528821,7.803922,7.632377,7.860131,0.828799,6474.000000,0.808986,...,4324.161851,4846.712526,8.428434,1.340511e+11,8.297693,0.858000,56.0,$\beta_{A}$,PAC-ML (Ours),4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1100,1.999739,2.383190e+10,21880.666623,2.822531,2.000000,1.999738,2.000000,0.734572,415.426174,0.733647,...,14858.302036,14955.821726,2.813701,2.376390e+10,2.806147,0.736197,2.0,$\beta_{D}$,PAC-ML (Ours),364
1101,1.999740,2.393668e+10,22086.413121,2.834639,2.000000,1.999739,2.000000,0.739506,416.223596,0.735738,...,14860.493631,15013.632708,2.802080,2.366379e+10,2.780842,0.743528,2.0,$\beta_{D}$,PAC-ML (Ours),365
1102,1.999739,2.380619e+10,22021.635956,2.818966,2.000000,1.999738,2.000000,0.738128,416.035794,0.733948,...,14862.342185,14964.578874,2.808674,2.372528e+10,2.794097,0.741622,2.0,$\beta_{D}$,PAC-ML (Ours),366
1103,1.999739,2.391429e+10,22030.847381,2.830550,2.000000,1.999737,2.000000,0.738377,414.872364,0.733478,...,14855.563431,14978.883989,2.811226,2.374233e+10,2.776940,0.741087,2.0,$\beta_{D}$,PAC-ML (Ours),367


key jobs_blocked_num_nodes_max -> 1130 vals
key jobs_blocked_num_edges_max -> 1130 vals
key jobs_blocked_original_demand_num_nodes_max -> 1130 vals
key jobs_blocked_max_acceptable_job_completion_time_mean -> 1130 vals
key jobs_blocked_original_demand_num_edges_mean -> 1130 vals
key jobs_blocked_total_dependency_size_max -> 1130 vals
key jobs_blocked_num_nodes_min -> 1130 vals
key jobs_blocked_original_demand_num_nodes_min -> 1130 vals
key jobs_blocked_original_demand_total_operation_memory_cost_min -> 1130 vals
key jobs_blocked_original_demand_total_dependency_size_min -> 1130 vals
key jobs_blocked_num_nodes_mean -> 1130 vals
key jobs_blocked_original_demand_total_dependency_size_max -> 1130 vals
key jobs_blocked_total_dependency_size_mean -> 1130 vals
key jobs_blocked_original_demand_total_operation_memory_cost_mean -> 1130 vals
key jobs_blocked_job_sequential_completion_time_min -> 1130 vals
key jobs_blocked_max_acceptable_job_completion_time_frac_mean -> 1130 vals
key jobs_blocked_m

,jobs_blocked_num_nodes_max,jobs_blocked_num_edges_max,jobs_blocked_original_demand_num_nodes_max,jobs_blocked_max_acceptable_job_completion_time_mean,jobs_blocked_original_demand_num_edges_mean,jobs_blocked_total_dependency_size_max,jobs_blocked_num_nodes_min,jobs_blocked_original_demand_num_nodes_min,jobs_blocked_original_demand_total_operation_memory_cost_min,jobs_blocked_original_demand_total_dependency_size_min,...,jobs_blocked_total_operation_memory_cost_mean,jobs_blocked_max_acceptable_job_completion_time_frac_min,jobs_blocked_max_acceptable_job_completion_time_max,jobs_blocked_original_demand_num_edges_max,jobs_blocked_original_demand_total_operation_memory_cost_max,jobs_blocked_job_sequential_completion_time_mean,jobs_blocked_total_operation_memory_cost_max,$\beta_{X}$,Agent,epoch_counter
0,100.496000,111.912000,100.496000,16556.840221,111.849333,1.594830e+10,100.406000,100.406000,1.564455e+10,1.590290e+10,...,1.566618e+10,0.542820,16717.139375,111.912000,1.568753e+10,29944.850467,1.568753e+10,$\beta_{A}$,PAC-ML (Ours),0
1,100.418000,111.834000,100.418000,16475.567237,111.725333,1.592421e+10,100.290000,100.290000,1.561901e+10,1.588348e+10,...,1.564112e+10,0.532670,16840.607368,111.834000,1.566630e+10,29956.443583,1.566630e+10,$\beta_{A}$,PAC-ML (Ours),1
2,100.460000,111.876000,100.460000,16568.687749,111.776000,1.591364e+10,100.318000,100.318000,1.560978e+10,1.586733e+10,...,1.563553e+10,0.543260,16827.603432,111.876000,1.565791e+10,29934.495483,1.565791e+10,$\beta_{A}$,PAC-ML (Ours),2
3,100.490000,111.906000,100.490000,16399.179505,111.784667,1.593757e+10,100.238000,100.238000,1.565911e+10,1.591568e+10,...,1.566847e+10,0.541420,16585.296061,111.906000,1.567417e+10,29944.377667,1.567417e+10,$\beta_{A}$,PAC-ML (Ours),3
4,100.450000,111.842000,100.450000,16439.400948,111.804667,1.594052e+10,100.386000,100.386000,1.564455e+10,1.590290e+10,...,1.566240e+10,0.528800,17155.459731,111.842000,1.568051e+10,29977.155150,1.568051e+10,$\beta_{A}$,PAC-ML (Ours),4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125,105.648352,117.153846,105.648352,19335.090189,114.177068,1.807224e+10,101.037736,101.037736,1.624270e+10,1.647070e+10,...,1.670262e+10,0.609780,19892.191797,117.153846,1.741721e+10,31039.012136,1.741721e+10,$\beta_{D}$,PAC-ML (Ours),373
1126,107.257143,119.457143,107.257143,18464.332733,113.139586,1.804458e+10,98.505051,98.505051,1.542213e+10,1.558575e+10,...,1.681750e+10,0.579899,19408.752224,119.457143,1.751934e+10,30767.594084,1.751934e+10,$\beta_{D}$,PAC-ML (Ours),374
1127,106.414414,118.405405,106.414414,19140.542133,113.512494,1.788752e+10,98.132075,98.132075,1.611146e+10,1.651075e+10,...,1.652663e+10,0.602432,19608.219703,118.405405,1.728443e+10,31174.360655,1.728443e+10,$\beta_{D}$,PAC-ML (Ours),375
1128,104.092593,115.092593,104.092593,19416.156854,114.307969,1.859787e+10,102.340000,102.340000,1.528164e+10,1.569513e+10,...,1.643438e+10,0.604352,19861.501365,115.092593,1.810359e+10,31331.393332,1.810359e+10,$\beta_{D}$,PAC-ML (Ours),376


In [8]:
%autoreload

# load baseline agent sweep
keys_to_ignore = ['valid/step_stats/'] # for ignoring data not needed for plotting
key_substrings_to_remove = ['valid/episode_stats/'] # for tidying up keys
verbose = False

_baseline_agent_to_episode_stats_dict, _baseline_agent_to_episode_completion_stats_dict, _baseline_agent_to_episode_blocked_stats_dict = defaultdict(list), defaultdict(list), defaultdict(list)
for baseline_agent in baseline_agent_to_beta_dist_to_sweep_path.keys():
    # load agent data for each dist
    hue = '$\\beta_{X}$' # grouping by beta dist
    dist_to_run = baseline_agent_to_beta_dist_to_sweep_path[baseline_agent]
    baseline_agent_to_episode_stats_dict, baseline_agent_to_episode_completion_stats_dict, baseline_agent_to_episode_blocked_stats_dict = load_ramp_cluster_environment_metrics_from_wandb_sweep(dist_to_run, keys_to_ignore=keys_to_ignore, key_substrings_to_remove=key_substrings_to_remove, verbose=verbose, hue=hue)
    
    # add agent column
    baseline_agent_to_episode_stats_dict['Agent'].extend([baseline_agent for _ in range(len(baseline_agent_to_episode_stats_dict[hue]))])
    baseline_agent_to_episode_completion_stats_dict['Agent'].extend([baseline_agent for _ in range(len(baseline_agent_to_episode_completion_stats_dict[hue]))])
    baseline_agent_to_episode_blocked_stats_dict['Agent'].extend([baseline_agent for _ in range(len(baseline_agent_to_episode_blocked_stats_dict[hue]))])
    
    # merge learner agent df into all learner agents df
    for key, val in baseline_agent_to_episode_stats_dict.items():
        _baseline_agent_to_episode_stats_dict[key].extend(val)
    for key, val in baseline_agent_to_episode_completion_stats_dict.items():
        _baseline_agent_to_episode_completion_stats_dict[key].extend(val)
    for key, val in baseline_agent_to_episode_blocked_stats_dict.items():
        _baseline_agent_to_episode_blocked_stats_dict[key].extend(val)

    
# remove leading underscore for readability        
baseline_agent_to_episode_stats_dict, baseline_agent_to_episode_completion_stats_dict, baseline_agent_to_episode_blocked_stats_dict = _baseline_agent_to_episode_stats_dict, _baseline_agent_to_episode_completion_stats_dict, _baseline_agent_to_episode_blocked_stats_dict
        
# convert dicts to dataframes
# for key, val in baseline_agent_to_episode_stats_dict.items():
#     print(f'key {key} -> {len(val)} vals')
baseline_agent_episode_stats_df = pd.DataFrame(baseline_agent_to_episode_stats_dict)
display(baseline_agent_episode_stats_df)

baseline_agent_episode_completion_stats_df = pd.DataFrame(baseline_agent_to_episode_completion_stats_dict)
display(baseline_agent_episode_completion_stats_df)

baseline_agent_episode_blocked_stats_df = pd.DataFrame(baseline_agent_to_episode_blocked_stats_dict)
display(baseline_agent_episode_blocked_stats_df)


Agent $\beta_{A}$ sweep <Sweep ong/ddls-scripts/xprqeqoh (FINISHED)> parameters:
	experiment.seed:
		{'values': [0, 1, 2]}
Loading data from 3 runs...
Loaded data for run 1 of 3 (<Run ong/ddls-scripts/7k4hh7ns (finished)>) in 0.401.
Loaded data for run 2 of 3 (<Run ong/ddls-scripts/nfm6liua (finished)>) in 0.399.
Loaded data for run 3 of 3 (<Run ong/ddls-scripts/4381qrv3 (finished)>) in 0.471.
Loaded data for agent $\beta_{A}$ sweep <Sweep ong/ddls-scripts/xprqeqoh (FINISHED)> (num_runs=3) in 1.271 s.

Agent $\beta_{B}$ sweep <Sweep ong/ddls-scripts/39cz3lq6 (FINISHED)> parameters:
	experiment.seed:
		{'values': [0, 1, 2]}
Loading data from 3 runs...
Loaded data for run 1 of 3 (<Run ong/ddls-scripts/hi7b14qz (finished)>) in 0.418.
Loaded data for run 2 of 3 (<Run ong/ddls-scripts/7e4o6iek (finished)>) in 0.397.
Loaded data for run 3 of 3 (<Run ong/ddls-scripts/0biqxyqw (finished)>) in 0.405.
Loaded data for agent $\beta_{B}$ sweep <Sweep ong/ddls-scripts/39cz3lq6 (FINISHED)> (num_runs

,dep_info_processed,num_jobs_arrived,blocking_rate,episode_end_time,mean_demand_total_throughput,compute_info_processed,num_jobs_blocked,mean_demand_compute_throughput,episode_time,demand_dep_info_processed,...,demand_total_info_processed,mean_dep_throughput,cluster_info_processed,mean_compute_throughput,mean_load_rate,mean_cluster_throughput,$\beta_{X}$,config,experiment.seed,Agent
0,9.048242e+13,1001,0.259740,1000000,2.328156e+07,1.155744e+13,260,1.155744e+07,1000000,1.172412e+13,...,2.328156e+13,9.048242e+07,1.020399e+14,1.155744e+07,3.157920e+07,1.020399e+08,$\beta_{A}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",2,Para$_{max}$
1,8.941673e+13,1001,0.262737,1000000,2.301142e+07,1.141525e+13,263,1.141525e+07,1000000,1.159617e+13,...,2.301142e+13,8.941673e+07,1.008320e+14,1.141525e+07,3.151933e+07,1.008320e+08,$\beta_{A}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",1,Para$_{max}$
2,9.079394e+13,1001,0.264735,1000000,2.344749e+07,1.162064e+13,265,1.162064e+07,1000000,1.182685e+13,...,2.344749e+13,9.079394e+07,1.024146e+14,1.162064e+07,3.146476e+07,1.024146e+08,$\beta_{A}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",0,Para$_{max}$
3,9.066860e+13,1001,0.261738,1000000,2.331935e+07,1.157350e+13,262,1.157350e+07,1000000,1.174585e+13,...,2.331935e+13,9.066860e+07,1.022421e+14,1.157350e+07,3.161611e+07,1.022421e+08,$\beta_{B}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",2,Para$_{max}$
4,9.160306e+13,1001,0.268731,1000000,2.368586e+07,1.172550e+13,269,1.172550e+07,1000000,1.196036e+13,...,2.368586e+13,9.160306e+07,1.033286e+14,1.172550e+07,3.159283e+07,1.033286e+08,$\beta_{B}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",1,Para$_{max}$
5,9.137898e+13,1001,0.258741,1000000,2.351176e+07,1.166970e+13,259,1.166970e+07,1000000,1.184206e+13,...,2.351176e+13,9.137898e+07,1.030487e+14,1.166970e+07,3.163812e+07,1.030487e+08,$\beta_{B}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",0,Para$_{max}$
6,9.066860e+13,1001,0.261738,1000000,2.331935e+07,1.157350e+13,262,1.157350e+07,1000000,1.174585e+13,...,2.331935e+13,9.066860e+07,1.022421e+14,1.157350e+07,3.161611e+07,1.022421e+08,$\beta_{C}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",2,Para$_{max}$
7,9.160306e+13,1001,0.268731,1000000,2.368586e+07,1.172550e+13,269,1.172550e+07,1000000,1.196036e+13,...,2.368586e+13,9.160306e+07,1.033286e+14,1.172550e+07,3.159283e+07,1.033286e+08,$\beta_{C}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",1,Para$_{max}$
8,9.185167e+13,1001,0.271728,1000000,2.366218e+07,1.172462e+13,272,1.172462e+07,1000000,1.193756e+13,...,2.366218e+13,9.185167e+07,1.035763e+14,1.172462e+07,3.162746e+07,1.035763e+08,$\beta_{C}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",0,Para$_{max}$
9,9.066860e+13,1001,0.261738,1000000,2.331935e+07,1.157350e+13,262,1.157350e+07,1000000,1.174585e+13,...,2.331935e+13,9.066860e+07,1.022421e+14,1.157350e+07,3.161611e+07,1.022421e+08,$\beta_{D}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",2,Para$_{max}$


,jobs_completed_max_partitions_per_op,job_communication_overhead_time,jobs_completed_original_demand_num_nodes,jobs_completed_num_edges,job_computation_overhead_time,jobs_completed_max_acceptable_job_completion_time_frac,jobs_completed_original_demand_total_operation_memory_cost,jobs_completed_max_acceptable_job_completion_time,jobs_completed_job_sequential_completion_time,jobs_completed_original_demand_num_edges,...,jobs_completed_total_dependency_size,job_completion_time_speedup,jobs_completed_total_operation_memory_cost,jobs_completed_num_mounted_channels,jobs_completed_mean_mounted_worker_utilisation_frac,jobs_completed_num_mounted_workers,$\beta_{X}$,config,experiment.seed,Agent
0,16.000000,4.474958,99.713514,24824.378378,1865.629267,0.552851,1.560399e+10,16467.144936,29755.115405,111.027027,...,1.221730e+11,15.663282,1.560399e+10,242.854054,3.656071e+05,16.081081,$\beta_{A}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",2,Para$_{max}$
1,16.000000,4.422981,100.358209,25142.721845,1877.668174,0.561927,1.546841e+10,16799.816123,29950.485075,111.751696,...,1.211649e+11,15.672534,1.546841e+10,242.626866,3.687270e+05,16.074627,$\beta_{A}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",1,Para$_{max}$
2,16.000000,4.506721,101.224490,25285.753741,1870.121754,0.549524,1.578809e+10,16421.602376,29827.598299,112.814966,...,1.233532e+11,15.665247,1.578809e+10,245.268027,3.766328e+05,16.149660,$\beta_{A}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",0,Para$_{max}$
3,16.000000,4.497680,99.994580,25022.878049,1875.050945,0.265298,1.567859e+10,7894.040075,29908.106369,111.292683,...,1.228269e+11,15.670423,1.567859e+10,245.008130,3.676363e+05,16.142276,$\beta_{B}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",2,Para$_{max}$
4,16.000000,4.580708,101.600547,25744.831737,1900.665455,0.277729,1.603666e+10,8388.113936,30323.351778,113.073871,...,1.252814e+11,15.687805,1.603666e+10,244.574555,3.854119e+05,16.129959,$\beta_{B}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",1,Para$_{max}$
5,16.000000,4.503673,100.267206,24966.862348,1863.715317,0.284251,1.571046e+10,8517.199163,29723.995951,111.680162,...,1.229960e+11,15.661498,1.571046e+10,244.512821,3.683223e+05,16.128205,$\beta_{B}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",0,Para$_{max}$
6,16.000000,4.497680,99.994580,25022.878049,1875.050945,0.518645,1.567859e+10,15484.348146,29908.106369,111.292683,...,1.228269e+11,15.670423,1.567859e+10,245.008130,3.676363e+05,16.142276,$\beta_{C}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",2,Para$_{max}$
7,16.000000,4.580708,101.600547,25744.831737,1900.665455,0.470109,1.603666e+10,14204.063406,30323.351778,113.073871,...,1.252814e+11,15.687805,1.603666e+10,244.574555,3.854119e+05,16.129959,$\beta_{C}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",1,Para$_{max}$
8,16.000000,4.613982,100.590659,25669.802198,1916.154737,0.467335,1.606645e+10,14342.471047,30575.263393,111.739011,...,1.258417e+11,15.700686,1.606645e+10,242.901099,3.801781e+05,16.082418,$\beta_{C}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",0,Para$_{max}$
9,16.000000,4.497680,99.994580,25022.878049,1875.050945,0.706694,1.567859e+10,21122.246735,29908.106369,111.292683,...,1.228269e+11,15.670423,1.567859e+10,245.008130,3.676363e+05,16.142276,$\beta_{D}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",2,Para$_{max}$


,jobs_blocked_original_demand_num_nodes,jobs_blocked_max_acceptable_job_completion_time,jobs_blocked_original_demand_total_dependency_size,jobs_blocked_max_acceptable_job_completion_time_frac,jobs_blocked_num_nodes,jobs_blocked_original_demand_num_edges,jobs_blocked_total_dependency_size,jobs_blocked_job_sequential_completion_time,jobs_blocked_total_operation_memory_cost,jobs_blocked_num_edges,jobs_blocked_original_demand_total_operation_memory_cost,$\beta_{X}$,config,experiment.seed,Agent
0,101.753846,16258.030558,1.613121e+10,0.529500,101.753846,113.276923,1.613121e+10,30475.478462,1.578636e+10,113.276923,1.578636e+10,$\beta_{A}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",2,Para$_{max}$
1,100.365019,16621.325618,1.637013e+10,0.566198,100.365019,111.798479,1.637013e+10,29802.749620,1.608153e+10,111.798479,1.608153e+10,$\beta_{A}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",1,Para$_{max}$
2,98.000000,16506.153096,1.534490e+10,0.546340,98.000000,108.871698,1.534490e+10,30271.997170,1.514234e+10,108.871698,1.514234e+10,$\beta_{A}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",0,Para$_{max}$
3,101.885496,8459.734977,1.600713e+10,0.279237,101.885496,113.633588,1.600713e+10,30056.933206,1.566171e+10,113.633588,1.566171e+10,$\beta_{B}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",2,Para$_{max}$
4,96.936803,8003.270801,1.470419e+10,0.276766,96.936803,108.078067,1.470419e+10,28904.516914,1.462869e+10,108.078067,1.462869e+10,$\beta_{B}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",1,Para$_{max}$
5,101.459459,8602.256811,1.601674e+10,0.284981,101.459459,112.961390,1.601674e+10,30457.497490,1.567420e+10,112.961390,1.567420e+10,$\beta_{B}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",0,Para$_{max}$
6,101.885496,16052.311901,1.600713e+10,0.531489,101.885496,113.633588,1.600713e+10,30056.933206,1.566171e+10,113.633588,1.566171e+10,$\beta_{C}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",2,Para$_{max}$
7,96.936803,13511.435147,1.470419e+10,0.468327,96.936803,108.078067,1.470419e+10,28904.516914,1.462869e+10,108.078067,1.462869e+10,$\beta_{C}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",1,Para$_{max}$
8,99.676471,13447.097085,1.478808e+10,0.471618,99.676471,111.602941,1.478808e+10,28480.446140,1.462172e+10,111.602941,1.462172e+10,$\beta_{C}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",0,Para$_{max}$
9,101.885496,21669.641542,1.600713e+10,0.718168,101.885496,113.633588,1.600713e+10,30056.933206,1.566171e+10,113.633588,1.566171e+10,$\beta_{D}$,"{""wandb"": {""value"": {""init"": {""entity"": ""ong"",...",2,Para$_{max}$
